In [26]:
!pip install transformers torch
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 35.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Cargar el modelo y el tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


## Cargar DB

In [29]:
import sqlite3
import pandas as pd
db_path = "/content/normativa_dgcye (1).db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('SELECT * FROM archivos ORDER BY id DESC')
filas = cursor.fetchall()

conn.close()

# Crear DataFrame
column_names = [column[0] for column in cursor.description]
resoluciones_df = pd.DataFrame(filas, columns=column_names)

In [30]:
texto = resoluciones_df["contenido"][10]

## Limpieza del texto

In [31]:
import re, unicodedata

def clean_text(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')

    # Eliminar números irrelevantes (no completos como años)
    #texto = re.sub(r'\b(19[5-9]\d|20[0-2]\d|2030)\b', '', texto)
    # Eliminar URLs
    texto = re.sub(r'https?://\S+|www\.\S+', ' ', texto)
    # Eliminar caracteres especiales
    texto = re.sub(r'[^\w\s.-]', ' ', texto)
    # Eliminar espacios extras
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def split_text(text, max_tokens=1024):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_chunk.append(word)
        current_length += len(word) + 1  # Consideramos un espacio por palabra
        if current_length >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def summarize_text(text, max_length=130, min_length=30):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def summarize_large_text(text, max_tokens=1024, max_length=130, min_length=30):
    chunks = split_text(text, max_tokens=max_tokens)
    summaries = [summarize_text(chunk, max_length=max_length, min_length=min_length) for chunk in chunks]
    return " ".join(summaries)

In [49]:
texto_limpio = clean_text(texto)

# Resumir el texto limpio
resumen = summarize_large_text(
    texto_limpio,
    max_tokens=1024,
    max_length=500,
    min_length=200
)

print("Resumen:")
print(resumen)



Resumen:
Un estudiante de la carrera de licenciatura en ciencias de la educacion de la universidad nacional de lujan plantea la situacion originada al trata r de inscribirse en los listados de emergencia para acceso a cargos docentes. No le fue permitido por no acreditar estudios de nivel secundario. Asegura que se le informo que no podia integrar el listado de emergentes y que tampoco podria aspirar a un ingreso com o docente en el sistema educativo provincial. Aplica una resolucion de aplicacion dependientes on las direcciones respectivas que la persona evaluada que ingresa y aprueba the carrera o un porcen taje de ella ha accedido a un nivell superior de conocimientos. Esta resolucion que sera desglosada para su archivo en la direccion de despacho la que en su lugar agregara copia autenticada de la misma comunicar al departamento mesa general de entradas y salidas. Notifica r al consejo general de cultura y educacion a todas las direccione s docentes y a the direccions de tribunales

## StopWords y lemmatización

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer
import spacy
import nltk
from spacy.lang.es.stop_words import STOP_WORDS
from nltk.corpus import stopwords as stopwords_en
import pandas as pd

In [34]:
nltk.download('wordnet')
nlp = spacy.load("es_core_news_sm")
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
stopwords_personalizadas = [
    "ejecutivo", "visto", "considerando", "expediente", "dirección", "general", "de", "cultura", "y",
    "educación", "la", "plata", "provincia", "buenos", "aires", "direccion", "educacion", "establecer",
    "resolución", "resolucion", "presente", "articulo", "of", "from", "that", "notificar", "the"
]

# Agregar palabras personalizadas a la lista de stopwords de spaCy
for word in stopwords_personalizadas:
    STOP_WORDS.add(word)

stopwords_es = STOP_WORDS
stopwords_en = set(stopwords_en.words('english'))
stopwords_combinadas = list(stopwords_es.union(stopwords_en))

# Verificar que las stopwords personalizadas están incluidas
faltantes = [word for word in stopwords_personalizadas if word not in stopwords_combinadas]

if not faltantes:
    print("Todas las palabras personalizadas están incluidas en las stopwords combinadas.")
else:
    print("Las siguientes palabras no están incluidas en las stopwords combinadas:", faltantes)

Todas las palabras personalizadas están incluidas en las stopwords combinadas.


In [36]:
# Función para lematizar el texto
def lemmatize_text(text):
    # Procesamos el texto con spaCy
    doc = nlp(text)
    # Extraemos las lemas (formas base) de las palabras
    lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop])

    return lemmatized_text


# Lematizar el texto
texto_lemmatizado = lemmatize_text(resumen)

print("Texto lematizado:", texto_lemmatizado)

Texto lematizado: estudiante carrera licenciatura ciencia universidad nacional lujar plantear situacion originado r inscribir él . peticionante acreditir alumno regular universidad ingresar articul 7 ley 24521 superior . permitir acreditar estudio nivel secundario . ley superior 7 citado permitir universidad reglamente s condición ingreso persona mayorar s 25 ano . consejo and expidiir referir alcance art .7 citado . ingreso carera nivel superior suplir requisito titulo secundario . consejo aprueba sesion fecha 16 -ix-99 dictamen comision asunto legal . directora and resolver 1 .- admitir inscripcion listado correspondiente aspirante docencia . - disponer efecto cumplimentacion establecido precedente as pirante deberar presentar certificacion acreditir ingreso carrera . refrendado vicepresidente 1 consejo . resolver aplicacion caso presente . comunicar departamento mesa entrada salida direccione s docente tribunal clasificacion .


## TF-IDF

In [37]:
def extract_keywords_tfidf(text, num_keywords=10):
    # Configurar el vectorizador TF-IDF con las stopwords personalizadas
    vectorizer = TfidfVectorizer(max_features=num_keywords, stop_words=stopwords_combinadas)

    # Transformar el texto
    tfidf_matrix = vectorizer.fit_transform([text])

    # Obtener las palabras clave ordenadas por importancia
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray().flatten()

    # Combinar palabras con sus puntuaciones y ordenarlas
    keywords = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
    return keywords

# Llamada a la función con el resumen
keywords_tfidf = extract_keywords_tfidf(texto_lemmatizado)
print("Palabras clave (TF-IDF):", keywords_tfidf)

keywords_df = pd.DataFrame(keywords_tfidf, columns=['Palabra', 'Puntuación'])

# Ordenar el DataFrame por puntuación en orden descendente
palabras_importantes_tfidf = keywords_df.sort_values(by='Puntuación', ascending=False)

# Mostrar las 10 palabras más importantes
print(palabras_importantes_tfidf.head(10))

Palabras clave (TF-IDF): [('consejo', 0.3872983346207417), ('ingreso', 0.3872983346207417), ('superior', 0.3872983346207417), ('universidad', 0.3872983346207417), ('acreditir', 0.2581988897471611), ('carrera', 0.2581988897471611), ('citado', 0.2581988897471611), ('ley', 0.2581988897471611), ('nivel', 0.2581988897471611), ('resolver', 0.2581988897471611)]
       Palabra  Puntuación
0      consejo    0.387298
1      ingreso    0.387298
2     superior    0.387298
3  universidad    0.387298
4    acreditir    0.258199
5      carrera    0.258199
6       citado    0.258199
7          ley    0.258199
8        nivel    0.258199
9     resolver    0.258199


## TF-IDF para bigramas y trigramas

In [38]:
# Crear el vectorizador de TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(3, 3), stop_words=stopwords_combinadas)

# Transformar el texto
X_tfidf = vectorizer.fit_transform([resumen])
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Sumar los valores de las columnas y ordenar
palabras_importantes_tfidf = df_tfidf.sum().sort_values(ascending=False)

print(palabras_importantes_tfidf.head(10))


16 ix 99                              0.101015
plantea situacion originada           0.101015
peticionante acredita alumno          0.101015
personas mayore 25                    0.101015
permitido acreditar estudios          0.101015
permite universidad reglamente        0.101015
originada inscribirse peticionante    0.101015
nivel superior suple                  0.101015
nivel secundario ley                  0.101015
nacional lujan plantea                0.101015
dtype: float64


In [40]:
# Crear el vectorizador de TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(2, 2), stop_words=stopwords_combinadas)

# Transformar el texto
X_tfidf = vectorizer.fit_transform([resumen])

# Convertir la matriz dispersa a un DataFrame
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Sumar los valores de las columnas y ordenar
palabras_importantes_tfidf = df_tfidf.sum().sort_values(ascending=False)

print(palabras_importantes_tfidf.head(10))

16 ix                    0.100504
precedente pirante       0.100504
pirante debera           0.100504
peticionante acredita    0.100504
personas mayore          0.100504
permitido acreditar      0.100504
permite universidad      0.100504
originada inscribirse    0.100504
nivel superior           0.100504
nivel secundario         0.100504
dtype: float64


In [41]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import unicodedata
import re
from concurrent.futures import ThreadPoolExecutor

# Tokenizer y modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

def split_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

def summarize_text(text, max_length=130, min_length=30):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_large_text(text, max_tokens=1024, max_length=130, min_length=30):
    chunks = split_text(text, max_tokens=max_tokens)
    summaries = []

    # Paralelización
    with ThreadPoolExecutor() as executor:
        summaries = list(executor.map(lambda chunk: summarize_text(chunk, max_length, min_length), chunks))

    return " ".join(summaries)

In [50]:
# Cargar modelo spaCy para español
nlp = spacy.load("es_core_news_sm")

# Función para lematizar texto
def lemmatize_text(text):
    # Procesamos el texto con spaCy
    doc = nlp(text)
    # Extraemos las lemas (formas base) de las palabras, excluyendo stopwords
    lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return lemmatized_text

# Función para extraer palabras clave usando TF-IDF
def extract_keywords_tfidf(text, num_keywords=10):
    # Configurar el vectorizador TF-IDF con las stopwords personalizadas
    vectorizer = TfidfVectorizer(max_features=num_keywords, ngram_range=(1, 1),stop_words=list(stopwords_combinadas))
    # Transformar el texto lematizado
    tfidf_matrix = vectorizer.fit_transform([text])
    # Obtener las palabras clave ordenadas por importancia
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray().flatten()
    # Combinar palabras con sus puntuaciones y ordenarlas
    keywords = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
    return keywords

# Función integrada que incluye lematización y extracción de palabras clave
def process_text_to_keywords(text, num_keywords=10):
    # Lematizar el texto
    lemmatized_text = lemmatize_text(text)
    # Extraer palabras clave con TF-IDF
    keywords_tfidf = extract_keywords_tfidf(lemmatized_text, num_keywords=num_keywords)
    # Convertir a DataFrame
    keywords_df = pd.DataFrame(keywords_tfidf, columns=['Palabra', 'Puntuación'])
    # Ordenar el DataFrame por puntuación en orden descendente
    return keywords_df.sort_values(by='Puntuación', ascending=False)


# Proceso completo
palabras_importantes_tfidf = process_text_to_keywords(resumen, num_keywords=10)

# Mostrar las palabras clave
print("Palabras clave (TF-IDF):")
print(palabras_importantes_tfidf)


Palabras clave (TF-IDF):
        Palabra  Puntuación
0       docente    0.612372
1       carrera    0.408248
2       listado    0.408248
3       acceder    0.204124
4     inscribir    0.204124
5      integrar    0.204124
6  licenciatura    0.204124
7         lugar    0.204124
8         lujar    0.204124
9          mesa    0.204124
